# Reactiv method for change detection
La méthode `List2reactiv` est une fonction Python qui prend en entrée une liste d'arrays complexes représentant des données temporelles et retourne trois valeurs :

1. `CV` : un tableau de nombres flottants compris entre 0 et 1, représentant le coefficient de variation pour l'axe temporel. Ce coefficient de variation est normalisé entre 0 (valeur moyenne théorique) et 1 (valeur moyenne théorique + écart-type théorique).

2. `K` : un tableau de nombres flottants compris entre 0 et 1, représentant la fraction temporelle où l'amplitude maximale est atteinte pendant la période d'observation.

3. `Amax` : un tableau d'arrays complexes non normalisés, représentant les valeurs d'amplitude maximale atteintes pendant la période d'observation.

La méthode commence par initialiser différentes variables, puis elle parcourt la liste d'arrays complexes, effectuant des calculs statistiques et identifiant les valeurs d'amplitude maximale.

Ensuite, elle calcule le coefficient de variation (CV) pour l'axe temporel en utilisant une formule théorique basée sur le nombre équivalent de looks (`L`). Si la valeur de `L` n'est pas spécifiée, elle est automatiquement estimée à partir des données.

Finalement, le coefficient de variation est normalisé et les trois résultats sont renvoyés.

L'utilisation de cette méthode permet d'analyser et de caractériser la variabilité temporelle des données complexes, en fournissant des mesures normalisées du coefficient de variation, de la fraction temporelle où l'amplitude maximale est atteinte, et des valeurs d'amplitude maximale elles-mêmes.

In [ ]:
from scipy.special import gamma 
import numpy as np

def List2reactiv(List, L=1):
    Nt = len(List)  # temporal dimension

    Imax = List[0]

    # Compute Coefficient of Variation    
    M1 = np.zeros_like(Imax, dtype=np.complex128)
    M2 = np.zeros_like(Imax, dtype=np.complex128)
    Kmax = np.zeros_like(Imax, dtype=np.float64)
    Amax = np.zeros_like(Imax, dtype=np.complex128)

    k = 0
    for count, im in enumerate(List):
        M1 = M1 + im
        M2 = M2 + im**2
        k = k + 1
        Kmax = (np.abs(im) > np.abs(Imax)) * count + (np.abs(im) < np.abs(Imax)) * Kmax 
        Amax = np.maximum(Amax, im)

    M1 = M1 / Nt
    M2 = M2 / Nt
    K = Kmax / Nt
            
    # Compute the Coefficient of variation       
    R = np.sqrt(M2 - M1**2) / M1 
    R[M1 == 0] = 0  # Remove possible Nan Output when Signal is zero
    R[M1.real < 0] = 0
    
    # Theoretical estimation of mean and std of the Coefficient of variation for the given Equivalent Number of Looks     
    if L is None:
        gam = R.real.mean()
        L = ((0.991936 + 0.067646 * gam - 0.098888 * gam**2 - 0.048320 * gam**3) /
             (0.001224 - 0.034323 * gam + 4.305577 * gam**2 - 1.163498 * gam**3))
    
    Rmean = np.sqrt((L * gamma(L)**2 / (gamma(L + 0.5)**2)) - 1)  # theoretical mean value
    num = (L * gamma(L)**4 * (4 * (L**2) * gamma(L)**2 - 4 * L * gamma(L + 1/2)**2 - gamma(L + 1/2)**2))
    den = (gamma(L + 1/2)**4 * (L * gamma(L)**2 - gamma(L + 1/2)**2))
    Rstd = 1 / 4 * num / den / np.sqrt(Nt)  # theoretical standard deviation value
    Rmax = Rmean + Rstd

    # Recast Coefficient of Variation between 0 (mean value) and 1 (max value)    
    CV = (R.real - Rmean.real) / (Rmax.real - Rmean.real)
    CV = (CV >= 0) * CV
    CV = CV * (CV < 1) + (CV >= 1)
    return CV, K, Amax


# Explanation of the coefficient of variation:

The CV array is scaled between 0 and 1. Each element in the array represents the coefficient of variation for the corresponding pixel or element in the images.

## Interpretation:
A value of 0 in the CV array indicates that the temporal variation for that pixel is at the theoretical mean value.
Values close to 1 indicate higher temporal variation compared to the mean.

In [ ]:
plt.imshow(CV, vmin=0.1, vmax=1)
plt.colorbar(label='Coefficient of Variation')
plt.title('Spatial Distribution of Coefficient of Variation')
plt.show()